## Объединяем Chapter`ы в один Booklet ...

In [1]:
import text_utils as te

In [2]:
import itertools

def interleave_lists(*lists):
    """ Returns plain list of lists`s elements.
        Elements are positioned by each list`s order 
        and order of lists. 
        Example:
        >>> lists = [[1,2,3],[11,22,33,44],[111,222,333,444,555]]
        >>> interleave_lists(*_lists)
        [1, 11, 111, 2, 22, 222, 3, 33, 333, 44, 444, 555]
        """
    tuples = list(itertools.zip_longest(*lists, fillvalue=None))
    cs = list(filter(None, [t for ts in tuples for t in ts]))
    return cs
    


In [3]:
from extract_terms import copyTerm

def merge_similar_terms(term_list, quiet=False):
    """ Простое слияние терминов с одинаковым .normalized полем. """ 
    res_list = []
    term_set = set(term_list)
    not quiet and print(len(term_set), 'unique terms out of', len(term_list), 'terms.')
    for i,t in enumerate(term_set):
        similar = [T for T in term_list if T==t]
        not quiet and i%1000==0 and print(i,'.',str(t),":", list(map(str,similar)))
        if len(similar) == 1:
            res_list.append(similar[0])
        elif len(similar) > 1:
            T = copyTerm(similar[0])
            T.lemmas = similar[0].lemmas
            T.count = sum([e.count for e in similar])
            res_list.append(T)
    return res_list

In [4]:
# STOPWORDS_PATH = '/home/user/dev/'+'texts/stopwords.txt'
STOPWORDS_PATH = './'+'text-corpus/stopwords.txt'

class Booklet(object):
    """ Книга, брошюра. Последовательность Глав (класс Chapter). """
    def __init__(self, ):
        self.chapter_list = []
        self.__term_candidates_cache = None
        
    def at(self, i):
        """ get chapter at index `i` """
        return self.chapter_list[i]

    def __len__(self):
        return self.size()

    def size(self):
        return len(self.chapter_list)
        
    def word_count(self):
        return sum([ch.word_count() for ch in self.chapter_list])
        
    def sentence_count(self):
        return sum([ch.size() for ch in self.chapter_list])
        
    def count_of(self, word):
        """return count of word(s) / term(s) found within sll chapters"""
        assert self.chapter_list
        return sum([ch.count_of(word) for ch in self.chapter_list])
        
    def get_stopwords(self):
        """return set of stopwords"""
        assert self.chapter_list
        return self.chapter_list[0].get_stopwords()
        
    def add_chapter(self, title='Untitled', text=None, user_data=None):
        
        et_obj = None
        
        if self.chapter_list:
            et_obj = self.chapter_list[0].get_extract_terms()
            start_pos = self.chapter_list[-1].sentence_list[-1].endpos
            start_pos.word += 1
            start_pos.sentence += 1
        else:
            start_pos = te.Position() # begin of text
            
        ch = te.Chapter(title=title, extract_terms_instance=et_obj)
        if text and type(text) is str:
            ch.load_text(text, start_position=start_pos)
        if len(ch.sentence_list) <= 0:
            return
        ch.user_data = user_data
        self.chapter_list.append(ch)

    def prepare_terms(self):
        """Extract nominal groups from the text for all chapters"""
        for ch in self.chapter_list[:]:
            print(ch.title, "...")
            try:
                ch.prepare_terms(stopwords_file=STOPWORDS_PATH, quiet=True)
            except AssertionError:
                self.chapter_list.remove(ch)

    def clear_cache(self):
        " reset cached data if any for all chapters "
        for ch in self.chapter_list:
            ch.clear_cache()
    
    def get_term_candidates(self, limit=None):
        """return term candidates list optionally
            cropped with limit for all chapters.
            Terms are positioned by total frequency (desc.). """
        if self.__term_candidates_cache:
            return self.__term_candidates_cache[:limit]

        cs = [ch.get_term_candidates() for ch in self.chapter_list]
        cs = interleave_lists(*cs)
        cs = merge_similar_terms(cs)[:limit]
        cs.sort(key=lambda t:-t.count)
        self.__term_candidates_cache = cs
        return cs

    def find_terms(self, substr, limit=None, exact_match=False):
        """ получить термы по полному совпадению или по части слова """
        ts = [ch.find_terms(substr,exact_match=exact_match)
              for ch in self.chapter_list]
        return interleave_lists(*ts)[:limit]
    
    def chapters_with_term(self, term, min_entries=1, min_score=0):
        """ Returns list(of tuples: (Chapter, score)).
            Найти главы (Chapter`ы) с указанным(-и) 
            термином(-ами) и отосртировать их по 
            убыванию плотности встречаемости термина.
            Показатель score показывает, насколько чаще терм встречается в
            главе, чем по коллекции.
            Если score больше 1, то термин упоминается в главе 
            чаще, чем по коллекции, если меньше 1 - то реже.
            Если порог min_entries указан, то в выдачу 
            попадут только те (Chapter`ы) с min_entries 
            и больше вхождений термина. """
        assert min_entries > 0
        chs = [(ch, ch.count_of(term)) for ch in self.chapter_list]
        chs = [tpl for tpl in chs  if tpl[1] >= min_entries]
#         Nword = self.count_of(term)
        Nword = sum([n for _,n in chs]) # этот термин по всей коллекции
        Nchpt = self.size()
        NwordsTotal = self.word_count()
        chs = [(ch, n/Nword*Nchpt * ch.word_count()/NwordsTotal*Nchpt) for ch,n in chs]
        
        chs = [tpl for tpl in chs  if tpl[1] >= min_score]
        chs.sort(key=lambda e:-e[1])
        return chs
        
    def df(self, term):
        """ DF - document frequency over collection
            Частота появления термина в документах коллекции """
        return len(self.chapters_with_term(term))
        
    def term_stdev_rank(self, term, parts):
        """
        Получить вес термина как среднеквадратическое отклонение его профиля, сжатого до `parts` фрагментов.
        Каждый chapter даёт целое число частей.
        Вес многословных терминов повышается кратно количеству слов.
        В случае, если `term` - это list / set терминов,
        в подсчёте среднеквадратического отклонения участвуют все термины,
        а длина термина определяется по первому термину из списка.
        """
        if type(term) in (list, set):
            t = next(iter(term))
        else:
            t = term
        assert hasattr(t, "normalized") and hasattr(t, "words")
        
        # find out 'parts' value for each chapter in the Booklet; Sum should be equal to parts param.
        N_sents = self.sentence_count() # total count of sentences in the Booklet
        parts4chs = [max(1, round(parts*ch.size()/N_sents)) for ch in self.chapter_list]
        
        full_profile = pd.Series()
        for i,ch in enumerate(self.chapter_list):
            t_series = ch.compress_profile(parts4chs[i], ch.profile4word_or_family(term) )
            full_profile = pd.concat([full_profile, t_series] )
#                                      ignore_index=True,
#                                      verify_integrity=False)
        
        rank = full_profile.std() * len(t.words)  # повышаем вес многословных терминов
        del t_series
        del full_profile
        return rank
    
    def rank_candidates_stdev(self,parts=None, min_count=1):
        parts = parts or self.size()
        assert parts > 0, "Positive values only allowed. Provided: %d" % parts

        ranks = []
        for t in self.get_term_candidates():
            if t.count < min_count or len(t.normalized) < 2:
                continue
            r = self.term_stdev_rank(t, parts)
            ranks.append( (t, r) )

        ranks.sort(key=lambda e:-e[1])        
        return ranks
    
    def relate_ranking_by_chapters(self, ranks):
        """ Соотносит термины из ранжированного списка терминов релевантным главам.
            Returns:
                list[ 
                     tuple( chapter_obj, rank_list[
                             tuple(Term, score)
                         ] )
                    ]
             ranks: list[ tuple(Term, score) ]       
        """
#         ranks = self.rank_candidates_stdev(parts=prt, min_count=5)

        ranks_by_ch = {}
        for t,t_sc in ranks:
            ch_sc_list = self.chapters_with_term(t, min_score=1)
            for ch,ch_sc in ch_sc_list:
                ch_i = self.chapter_list.index(ch)
#                 print(ch_i)
                elem_tuple = (t,t_sc*ch_sc)
                if ch_i in ranks_by_ch:
                    ranks_by_ch[ch_i].append(elem_tuple)
                else:
                    ranks_by_ch[ch_i] = [elem_tuple]
        res_list = []
        for i in ranks_by_ch:
            # print(i, bl0.at(i).title)
            ranks_by_ch[i].sort(key=lambda e:-e[1])
            res_list.append( (self.at(i), ranks_by_ch[i]) )
        ranks_by_ch.clear()
        return res_list

In [5]:
bl = Booklet()

In [6]:
bl.add_chapter(title="Ch1", text="Веселый забег на свежем воздухе? "*50)
bl.add_chapter(title="Ch2", text="Мноогобещающий старт.Вдохни свежего воздуха! "*50)


In [7]:
bl.sentence_count()

100

In [8]:
bl.prepare_terms()

Ch1 ...


/home/user/miniconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Ch2 ...


In [9]:
bl.at(0).get_term_candidates()[0].normalized

'свежий воздух'

In [10]:
cl = bl.get_term_candidates()
cl

2 unique terms out of 3 terms.
0 . весёлый забег : ['весёлый забег']


In [11]:
print([(t,t.count) for t in cl])

[(<rutermextract.term_extractor.Term object at 0x7f953444ba20>, 100), (<rutermextract.term_extractor.Term object at 0x7f9534459eb8>, 50)]


In [12]:
for t in bl.get_term_candidates():
#     print(t.normalized)
    print(str(t), '-', t.count)
        

свежий воздух - 100
весёлый забег - 50


In [13]:
ch_t_sc = bl.relate_ranking_by_chapters(bl.rank_candidates_stdev(20,min_count=7))

for ch,ts in ch_t_sc:
    print(ch.title, "---", [str(t) for t,_ in ts])
    

Ch1 --- ['весёлый забег', 'свежий воздух']
Ch2 --- ['свежий воздух']


In [14]:
for pts in range(1,10+1):
    for i in [0,1,2]:
        print(pts,':', bl.term_stdev_rank(bl.get_term_candidates()[i], pts))

1 : 57.735026918962575
1 : 50.0


IndexError: list index out of range

In [ ]:
stdranks = [bl.term_stdev_rank(bl.get_term_candidates()[2], pts)
           for pts in range(1,10+1)]
pd.Series(stdranks).plot()

### Объединяем статьи из КиберЛенинки

In [13]:
from eval_utils import load_data

In [14]:
grouped_cybl_articles_paths = [
    ("text-corpus/cyberleninka/grouped_articles_min%dkeywords.pkl"
     % i) for i in range(1,2+1)
]

grouped_cybl_articles = [load_data(p) for p in grouped_cybl_articles_paths]
[len(g) for g in grouped_cybl_articles], "всего групп в выборках"

([51, 35], 'всего групп в выборках')

In [15]:
[sum([len(g) for g in gs])  for gs in grouped_cybl_articles], "всего статей в выборках"

([116, 82], 'всего статей в выборках')

In [16]:
g0 = [a for gr0 in grouped_cybl_articles[1] for a in gr0]

len(g0), g0[0].keys()

(82,
 dict_keys(['status', 'number', 'title', 'topic', 'year', 'Authors', 'url', 'keywords', 'abstract', 'text', 'similar', 'article-ID']))

In [17]:
bl0 = Booklet()

for a in g0[:5]:
    bl0.add_chapter(title=a['title'],text=a['text'], user_data=a)

In [18]:
%%time

bl0.prepare_terms()

Структурная диагностическая модель компьютерной сети ...


/home/user/miniconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Метод поиска дефектов в сегменте компьютерной сети на основе применения ее структурной и функциональной модели ...
Полная структурная оценка защищенности ИС ...
Оценка защищенности ИС ...
Микропроцессорная реализация наблюдателей состояния и нагрузки для электромеханотронных модулей ...
CPU times: user 7.75 s, sys: 134 ms, total: 7.88 s
Wall time: 7.86 s


In [19]:
print(bl0.size(), 'chapters',
 bl0.sentence_count(), "sentences",
 bl0.word_count(), "words",
)

5 chapters 291 sentences 6069 words


In [20]:
%%time

for t in bl0.get_term_candidates():
    print(t.count,
          t.normalized.ljust(20),
#           ' '.join([str(w) for w in t.words]), 
          sep='\t')
    if t.count < 4:
        break

994 unique terms out of 1254 terms.
0 . новые факторы сложности : ['новые факторы сложности']
39	л                   
33	компьютерная сеть   
26	рис                 
15	уязвимость          
13	система             
12	использование       
12	источник тестов     
11	информационная система
10	узел                
10	тестовое воздействие
10	сеть                
9	время               
9	функциональная модель
9	определение         
8	ас                  
8	совокупная уязвимость
8	сервер              
8	число               
7	модель              
7	таблица неисправностей
7	оценка              
7	область             
6	сетевое компонент   
6	реакция             
6	размерность         
6	рисунок             
6	суис                
6	запрос              
6	тест                
5	наблюдатели состояния
5	явная адресуемая неисправность
5	следующий образ     
5	количество          
5	адресный тест       
5	м                   
5	номер               
5	настоящее           
5	подозреваемые неисправнос

In [404]:
%%time
"Просто Тест"

PRT = 20
ranks = bl0.rank_candidates_stdev(parts=PRT, min_count=5)

for t,r in ranks:
    print(t.count,
          t.normalized.ljust(30),
          r,
          sep='\t' 
         )


14	данные                        	18.604713880208024
27	системы                       	17.256804053349313
134	система                       	16.769467619688694
6	обучение                      	14.411432285896275
92	модель                        	13.142220313513553
6	задачи                        	12.39925458475712
7	операции                      	12.16122115174589
27	выполнение                    	11.95281776433384
26	операция                      	11.833346212964033
16	курс                          	10.794430261965315
25	характеристика                	10.305981499624371
7	характеристики                	10.305981499624371
36	задача                        	10.141799948465277
41	множество                     	9.998234849128544
45	выход                         	9.97531762160571
25	этап                          	9.84271289077089
14	цикл                          	9.807079095903822
127	время                         	9.760071569929572
29	процесс                       	9.634260603931374
17	про

14	случайная величина            	1.83075442106412
12	обнаружение                   	1.8283251519784256
15	графика                       	1.821304059159134
10	график                        	1.821304059159134
8	графики                       	1.821304059159134
12	главное                       	1.8189865029016699
22	пример                        	1.8058573985652362
16	основание                     	1.7976713879013198
46	зависимость                   	1.789469909369487
8	печатный документ             	1.7858836834949385
14	контроль                      	1.7832609639879924
9	прошивка                      	1.7817163843710597
11	назначение                    	1.7560122876493822
8	кабельный канал               	1.7340139349066215
12	ячейка                        	1.7328340556172088
8	автоматизация                 	1.7300229336723971
13	составной канал               	1.7294942736181222
8	множество вершин              	1.729494273618122
23	обеспечение                   	1.7268314633034885
7	затр

12	различные типы                	0.906569489846938
9	концепция                     	0.9037559559556276
11	сохранение                    	0.9032335662069422
19	км                            	0.899032583959363
15	пределы                       	0.8974672107085315
5	обработка информации          	0.8957454337064186
20	цп                            	0.8949986678961537
17	данная работа                 	0.8867870923165897
5	логические операции           	0.8780061438246907
6	попытка                       	0.8768855228909038
10	гость                         	0.8670069674533107
6	данные дистанционное зондирование	0.8667914591343866
7	дисперсия                     	0.8633815660607825
5	задание                       	0.8629714734635204
12	диаметр                       	0.8575192318766668
31	интернет                      	0.8526550586066746
7	полость                       	0.8511661270346755
5	маркировка                    	0.8511661270346755
6	корректирующие мероприятия    	0.8500101219826501
5	

5	аналоги                       	0.22472533218632404
5	проф                          	0.21999374264811822
7	канды                         	0.21891841037963553
5	итог                          	0.21891841037963553
8	техн                          	0.2174763631636933
6	выделение                     	0.2174763631636933
5	дополнение                    	0.21747636316369323
7	примечание                    	0.20781203215397462
5	бхв-петербург                 	0.19876709464958617
5	следствие                     	0.19757626758863936
5	сапр                          	0.19757626758863936
5	владимирович                  	0.19757626758863936
5	комментарий                   	0.18667509462642745
5	отдельность                   	0.17498419650513775
5	предупреждение                	0.17363033325248423
6	рффи                          	0.16111704461321513
5	доцент                        	0.16111704461321513
5	наглядность                   	0.1474129578349874
14	ы82бм                         	0.0
10	т2ик    

In [406]:
len(ranks)

1015

In [298]:
rankings = []
topN = 25

for prt in range(10,60+1,10):
    print("parts =" , prt, ":")
    ranks = bl0.rank_candidates_stdev(parts=prt, min_count=5) # [:topN]
    rankings.append(ranks)
    for t,r in ranks[:topN]:
        print(t.count,
              t.normalized.ljust(30),
              r,
              sep='\t' 
             )


parts = 10 :
10	модель                        	24.109578902830197
9	системы                       	23.11841035702709
9	система                       	22.94853495128298
12	цикл                          	22.279561175073585
12	обработка                     	21.336963833709465
9	выполнение                    	21.16019485150943
12	этап                          	20.724752944616363
13	выход                         	20.26793286361741
6	фрагмент                      	17.377852099432296
9	множество                     	15.787369180488106
7	иерархическая система         	13.281566172707194
5	фаза обработки                	13.281566172707194
23	фильтр                        	13.27845275972723
8	длительность обработки        	12.332882874656677
6	схема обработки               	12.332882874656677
6	задача                        	12.236033377713254
11	процессоры                    	10.118243234566274
9	ошибка                        	9.984603532054125
11	работа                        	9.59580570767703

In [299]:
cropped_ranks = [rl[:40] for rl in rankings]
rs = interleave_lists(*cropped_ranks)

for t,r in rs:
    print(t.count,
          t.normalized.ljust(30),
          r, sep='\t')


10	модель                        	24.109578902830197
9	системы                       	17.727235488050034
10	модель                        	14.413426376243148
10	модель                        	10.829928731152348
10	модель                        	10.08515331863217
10	модель                        	8.917286407316384
9	системы                       	23.11841035702709
9	система                       	17.584049035640366
9	системы                       	12.966716962757786
9	системы                       	10.038416830466344
6	фрагмент                      	8.874566834946735
9	системы                       	7.799309286860772
9	система                       	22.94853495128298
10	модель                        	17.28639673833509
9	система                       	12.881792565871605
9	система                       	9.978394688353623
9	системы                       	8.501502551603489
9	система                       	7.757461449321883
12	цикл                          	22.279561175073585
12	цикл        

In [300]:
simpl_cr_rs = [[(t.count,t.normalized) for t,r in rs] for rs in cropped_ranks]

In [301]:
rs_sets = [set(rs) for rs in simpl_cr_rs]
u_set = set(rs_sets[0])
i_set = set(rs_sets[0])
for rs_set in rs_sets:
    u_set.update(rs_set)
    i_set.intersection_update(rs_set)

len(u_set), u_set, len(i_set), i_set

(46,
 {(5, 'безопасность'),
  (5, 'разработка'),
  (5, 'сторона'),
  (5, 'фаза обработки'),
  (6, 'задача'),
  (6, 'процессы'),
  (6, 'схема обработки'),
  (6, 'фрагмент'),
  (6, 'электрические параметры питания узлов'),
  (7, 'иерархическая система'),
  (7, 'процесс'),
  (7, 'топология'),
  (8, 'алгоритм'),
  (8, 'длительность обработки'),
  (8, 'документ'),
  (8, 'печатный документ'),
  (8, 'реализация'),
  (9, 'выполнение'),
  (9, 'газовая пружина'),
  (9, 'множество'),
  (9, 'ошибка'),
  (9, 'система'),
  (9, 'системы'),
  (10, 'метод'),
  (10, 'модель'),
  (11, 'значение'),
  (11, 'объект'),
  (11, 'процессоры'),
  (11, 'работа'),
  (11, 'результат'),
  (12, 'вероятность'),
  (12, 'информация'),
  (12, 'обработка'),
  (12, 'цикл'),
  (12, 'этап'),
  (13, 'выход'),
  (13, 'импульсный шум'),
  (13, 'количество'),
  (14, 'решение'),
  (17, 'оценка'),
  (21, 'время'),
  (23, 'фильтр'),
  (29, 'анализ'),
  (35, 'использование'),
  (35, 'рисунок'),
  (70, 'рис')},
 36,
 {(5, 'безопаснос

In [302]:
u_set.difference(i_set)

{(5, 'разработка'),
 (5, 'сторона'),
 (6, 'электрические параметры питания узлов'),
 (7, 'процесс'),
 (8, 'печатный документ'),
 (9, 'газовая пружина'),
 (12, 'информация'),
 (13, 'импульсный шум'),
 (13, 'количество'),
 (35, 'рисунок')}

####  Распределение отобранных терминов по отдельным Главам

In [26]:
PRT = 40

ch_t_sc = bl0.relate_ranking_by_chapters(bl0.rank_candidates_stdev(PRT,min_count=3))

In [27]:
for ch,ts in ch_t_sc[:5]:
    print(ch.title, "\n---\n", [str(t) for t,_ in ts[:20]])
    

Обеспечение жизнеспособности информации во времени при ее обработке в СППР 
---
 ['отклонение', 'требование', 'требования', 'надёжность', 'состояние', 'состояния', 'компонента', 'изменение', 'отрезка', 'выполнение', 'жизнеспособная система', 'граница', 'жизнеспособность', 'обновление', 'сложная система', 'элемент', 'элементы', 'структура', 'достоверность', 'период']
Интеграция данных и язык запросов в масштабных информационных инфраструктурах 
---
 ['интеграция', 'распределенный запрос', 'запрос', 'запросы', 'бд', 'положение', 'глобальная схема', 'распределенные системы', 'виртуальная интеграция', 'способ', 'способ интеграции', 'массовая интеграция', 'интегрируемые бд', 'технология', 'технологии', 'проблема', 'данные', 'выполнение', 'инфраструктура', 'результат']
Анализ факторов выбора системы управления данными 
---
 ['изделие', 'подготовка', 'состав изделия', 'внедрение', 'изменение', 'требование', 'требования', 'состав', 'предприятие', 'среда', 'структура', 'станция', 'электронная с

In [28]:
import eval_utils as eu

In [359]:
"RELOAD MODULE FROM CHANGED FILE"
import eval_utils as eu
from imp import reload         # Импортировать функцию reload (в 3.0)
reload(eu)                 # Загрузить обновленный модуль

# make_eval_data = eval_utils.make_eval_data


<module 'eval_utils' from '/home/user/dev/concepts/eval_utils.py'>

In [412]:
ch.user_data.keys()

dict_keys(['status', 'number', 'title', 'topic', 'year', 'Authors', 'url', 'keywords', 'abstract', 'text', 'similar', 'article-ID'])

In [30]:
import text_utils as te

In [47]:
def text2collocations(text, max_N = 3, uniqualize=False):
    """ Из слов текста длинее 1 буквы строится набор кортежей-коллокаций слов, 
        каждая коллокация длиной от 1 до max_N слов.
        Предварительно текст делится на предложения, т.обр. 
        коллокации не переходят через границу предложений.
        Если параметр uniqualize истинен, то коллокации-дубликаты 
        будут удалены, а порядок кортежей в вохвращаемом списке станет случайным. 
        Пример:
        
        text2collocations("One,two, Three, Четыре, пять? Я иду ИСКАТЬ! OnE,TWO, ...", max_N=5, uniqualize=1)
        """
    w_tuple_list = []
    sentences = te.extract_sentences_from_text(text)
    s_words = [
        [w.lower() for w in te.sentence_word_re.findall(line) if len(w)>=2] # минимум 2 буквы в слове
        for line in sentences
    ]
    sentences.clear()
    max_N = min(max_N, max([len(ws) for ws in s_words]))
    for n in range(1, max_N+1):
        for w_list in s_words:
            for p in range(len(w_list)+1-n):
                w_tuple_list.append( tuple(w_list[p:p+n]) )
    if uniqualize:
        w_tuple_list = list(set(w_tuple_list))
    return w_tuple_list
    
    
len(text2collocations("One,two, Three, Четыре, пять? Я иду ИСКАТЬ! OnE,TWO, ...", max_N=5, uniqualize=1))

18

In [32]:
def ch2test_sets(ch):
    """ Построение из "article"-словаря КиберЛенинки тестового набора для оценки качества извлечения терминов """
    return {
        "title"    : text2collocations(ch.user_data["title"], max_N=4, uniqualize=True),
        "keywords" : ch.user_data["keywords"],
        "abstract" : text2collocations(ch.user_data["abstract"], max_N=5, uniqualize=True),
#         "topic"    : text2collocations(ch.user_data["topic"], max_N=4, uniqualize=True),
    }

In [33]:
raED_list = []

for ch,t_scs in ch_t_sc:
    ts_std = [str(t) for t,_ in t_scs]
    ts_freq = [str(t) for t in ch.get_term_candidates(limit=100)]
    red = eu.RankingEvalData(ch.user_data['article-ID'], 
                             rankings={
                                 "stdev-%d"%PRT : ts_std,
                                 "freq"        : ts_freq
                             }, 
                             test_sets=ch2test_sets(ch))
    raED_list.append(red)


In [34]:
len(raED_list)

82

In [35]:
eu.dump_data(raED_list, "result/cylen_%d_of%d_[Raed].pkl" % (len(raED_list), len(g0)))

saved result/cylen_82_of82_[Raed].pkl
